# Class Workbook

## In class activity

In [1]:
import numpy as np
import pandas as pd
import math
from matplotlib.pyplot import subplots

#import statsmodels.api as sm
from plotnine import *
import plotly.express as px
import statsmodels.formula.api as sm
#import ISLP as islp

### Ames Housing data

Please take a look at the Ames Hoursing data.

In [2]:
ames_raw=pd.read_csv("ames_raw.csv")


### Questions

Use data of `ames_raw` up to 2008 predict the housing price for the later years.

In [4]:
ames_raw_2009, ames_raw_2008= ames_raw.query('`Yr Sold`>=2008').copy(), ames_raw.query('`Yr Sold` <2008').copy()


Use the same loss function calculator.

In [5]:
def calc_loss(prediction,actual):
  difpred = actual-prediction
  RMSE =pow(difpred.pow(2).mean(),1/2)
  operation_loss=abs(sum(difpred[difpred<0]))+sum(0.1*actual[difpred>0])
  return RMSE,operation_loss

Here are few rules:

- You are not allowed to use the test data.
- Try 3 of the regularization methods discussed in Ch6.
- You should use a resampling method that is most appropriate for choosing the hyper parameters.

Your code:

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# Setting up the grid search parameters for each regularization method
ridge_params = {'alpha': [0.1, 1.0, 10.0, 100.0]}
lasso_params = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0]}
elastic_net_params = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0], 'l1_ratio': [0.2, 0.5, 0.8]}

# Creating a pipeline for each model with preprocessing steps
ridge_pipeline = Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler()), ('ridge', Ridge())])
lasso_pipeline = Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler()), ('lasso', Lasso(max_iter=10000))])
elastic_net_pipeline = Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler()), ('elastic_net', ElasticNet(max_iter=10000))])

# Setting up GridSearchCV for each model
ridge_search = GridSearchCV(ridge_pipeline, [{'ridge__alpha': ridge_params['alpha']}], cv=5, scoring='neg_root_mean_squared_error')
lasso_search = GridSearchCV(lasso_pipeline, [{'lasso__alpha': lasso_params['alpha']}], cv=5, scoring='neg_root_mean_squared_error')
elastic_net_search = GridSearchCV(elastic_net_pipeline, [{'elastic_net__alpha': elastic_net_params['alpha'], 'elastic_net__l1_ratio': elastic_net_params['l1_ratio']}], cv=5, scoring='neg_root_mean_squared_error')

# Fit the models
X_train_imputed = X_train.fillna(X_train.mean())
ridge_search.fit(X_train_imputed, y_train)
lasso_search.fit(X_train_imputed, y_train)
elastic_net_search.fit(X_train_imputed, y_train)

# Best parameters and scores for each model
ridge_best_params, ridge_best_score = ridge_search.best_params_, ridge_search.best_score_
lasso_best_params, lasso_best_score = lasso_search.best_params_, lasso_search.best_score_
elastic_net_best_params, elastic_net_best_score = elastic_net_search.best_params_, elastic_net_search.best_score_

ridge_best_params, ridge_best_score, lasso_best_params, lasso_best_score, elastic_net_best_params, elastic_net_best_score



({'ridge__alpha': 100.0},
 -39871.099152521994,
 {'lasso__alpha': 10.0},
 -39990.03716316481,
 {'elastic_net__alpha': 0.1, 'elastic_net__l1_ratio': 0.2},
 -39864.43798665829)

Your answer:

~~~
Please write your answer in full sentences.


~~~

- For each of the models you've run, can you interpret which variable impacts the outcome the most?
Are the results consistent across different methods?

Your code:

In [19]:
# Extracting the best estimator (fitted model) for each regularization method
ridge_best_model = ridge_search.best_estimator_.named_steps['ridge']
lasso_best_model = lasso_search.best_estimator_.named_steps['lasso']
elastic_net_best_model = elastic_net_search.best_estimator_.named_steps['elastic_net']

# Extracting coefficients from each model
ridge_coefficients = ridge_best_model.coef_
lasso_coefficients = lasso_best_model.coef_
elastic_net_coefficients = elastic_net_best_model.coef_

# Creating a DataFrame to display the coefficients for comparison
coefficients_df = pd.DataFrame({
    'Feature': features,
    'Ridge': ridge_coefficients,
    'Lasso': lasso_coefficients,
    'ElasticNet': elastic_net_coefficients
})

# Identifying the most impactful variable for each model
coefficients_df['Ridge_Max_Impact'] = coefficients_df['Ridge'].abs().idxmax()
coefficients_df['Lasso_Max_Impact'] = coefficients_df['Lasso'].abs().idxmax()
coefficients_df['ElasticNet_Max_Impact'] = coefficients_df['ElasticNet'].abs().idxmax()

coefficients_df



,Feature,Ridge,Lasso,ElasticNet,Ridge_Max_Impact,Lasso_Max_Impact,ElasticNet_Max_Impact
0,Overall Qual,32699.468626,35793.654720,32556.046266,0,0,0
1,Gr Liv Area,25929.347735,27209.531154,25861.696472,0,0,0
2,Garage Cars,6245.615533,3845.035382,6328.410091,0,0,0
3,Garage Area,8312.021196,9036.159586,8306.434760,0,0,0
4,Total Bsmt SF,12921.635348,12282.943990,12943.116255,0,0,0


In [20]:
# Correcting the method to identify the most impactful variable
most_impactful_ridge = features[np.argmax(abs(ridge_coefficients))]
most_impactful_lasso = features[np.argmax(abs(lasso_coefficients))]
most_impactful_elastic_net = features[np.argmax(abs(elastic_net_coefficients))]

# Summary of the most impactful variables for each regularization method
impact_summary = {
    'Ridge': most_impactful_ridge,
    'Lasso': most_impactful_lasso,
    'ElasticNet': most_impactful_elastic_net
}

impact_summary


{'Ridge': 'Overall Qual',
 'Lasso': 'Overall Qual',
 'ElasticNet': 'Overall Qual'}

Your answer:

~~~
Please write your answer in full sentences.


~~~

- Try these methods with bootstrap, are the results consistent?

Your code:

In [21]:
from sklearn.utils import resample

# Number of bootstrap samples
n_bootstrap_samples = 100

# Dictionary to store the most impactful variables for each model and bootstrap sample
impact_variables = {'Ridge': [], 'Lasso': [], 'ElasticNet': []}

# Perform bootstrap sampling and model training/evaluation
for _ in range(n_bootstrap_samples):
    # Generate a bootstrap sample
    X_bootstrap, y_bootstrap = resample(X_train_imputed, y_train, replace=True)
    
    # Train and evaluate Ridge model
    ridge_model = Ridge(alpha=ridge_best_params['ridge__alpha'])
    ridge_model.fit(X_bootstrap, y_bootstrap)
    ridge_impact_variable = features[np.argmax(abs(ridge_model.coef_))]
    impact_variables['Ridge'].append(ridge_impact_variable)
    
    # Train and evaluate Lasso model
    lasso_model = Lasso(alpha=lasso_best_params['lasso__alpha'], max_iter=10000)
    lasso_model.fit(X_bootstrap, y_bootstrap)
    lasso_impact_variable = features[np.argmax(abs(lasso_model.coef_))]
    impact_variables['Lasso'].append(lasso_impact_variable)
    
    # Train and evaluate Elastic Net model
    elastic_net_model = ElasticNet(alpha=elastic_net_best_params['elastic_net__alpha'], l1_ratio=elastic_net_best_params['elastic_net__l1_ratio'], max_iter=10000)
    elastic_net_model.fit(X_bootstrap, y_bootstrap)
    elastic_net_impact_variable = features[np.argmax(abs(elastic_net_model.coef_))]
    impact_variables['ElasticNet'].append(elastic_net_impact_variable)

# Summary of the most frequently identified impactful variable for each model type
impact_summary_bootstrap = {model: max(set(variables), key=variables.count) for model, variables in impact_variables.items()}

impact_summary_bootstrap



{'Ridge': 'Overall Qual',
 'Lasso': 'Overall Qual',
 'ElasticNet': 'Overall Qual'}

Your answer:

~~~
Please write your answer in full sentences.


~~~
After performing bootstrap sampling and evaluating the Ridge, Lasso, and Elastic Net models on 100 bootstrap samples, the results are consistent with the earlier findings: Overall Qual (Overall Quality) is identified as the most impactful variable for the  across all three regularization methods.SalePrice

- Based on all the models you've run, which result predicts the best?  Which one is most easy to understand?  Which method gives the most stable results?

Your code:

Your answer:

~~~
Please write your answer in full sentences.


~~~
Best Prediction: Elastic Net slightly edged out with the best RMSE score in our initial model evaluation, indicating potentially the best predictive performance among the three.
Ease of Understanding: Lasso might be the easiest to understand due to its feature selection capability, making it more straightforward to interpret the most impactful variables.
Stability: All three methods showed stability in the bootstrap analysis, consistently identifying the same key variable.
Each method has its advantages, and the choice among them can depend on the specific goals of the analysis, the nature of the data, and the importance of interpretability versus predictive accuracy.

## Problem Set

### Best Subset

In this exercise, we will generate simulated data, and will then use this data to perform best subset selection.

(a) Use the `normal()` function to generate a predictor $X$ of length $n = 100$, as well as a noise vector $\epsilon$ of length $n = 100$.

Your code:

In [22]:
import numpy as np

# Set the seed for reproducibility
np.random.seed(0)

# Generate predictor X and noise vector epsilon
n = 100
X = np.random.normal(size=n)
epsilon = np.random.normal(size=n)

X[:10], epsilon[:10]  # Displaying the first 10 elements of each to verify



(array([ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ,  1.86755799,
        -0.97727788,  0.95008842, -0.15135721, -0.10321885,  0.4105985 ]),
 array([ 1.8831507 , -1.34775906, -1.270485  ,  0.96939671, -1.17312341,
         1.94362119, -0.41361898, -0.74745481,  1.92294203,  1.48051479]))

(b) Generate a response vector $Y$ of length $n = 100$ according to the model $$\mathbf{y} = \boldsymbol{\beta}_0 + \beta_1X + \beta2X^2 + \beta_3X^3 + \epsilon$$, where $\beta_{0}$ , $\beta_{1}$, $\beta_{2}$, and $\beta_{3}$ are constants of your choice.

Your code:

In [23]:
# Constants for the model
beta_0 = 2
beta_1 = 3
beta_2 = -1.5
beta_3 = 0.5

# Generate the response vector Y
Y = beta_0 + beta_1*X + beta_2*X**2 + beta_3*X**3 + epsilon

Y[:10]  # Displaying the first 10 elements of Y to verify


array([ 7.25224692,  1.64456162,  2.69761717,  7.78611007,  4.45470032,
       -0.88750593,  3.51145148,  0.76237634,  3.59675442,  4.49403524])

(c) Use the `regsubsets()` function to perform best subset selection in order to choose the best model containing the predictors $X$, $X^{2},\dots,X^{10}$. What is the best model obtained according to $C_p$ , BIC, and adjusted $R^2$ ? Show some plots to provide evidence for your answer, and report the coefficients of the best model obtained. Note you will need to use the `data.frame()` function to create a single data set containing both $X$ and $Y$.

Your code:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import itertools
import pandas as pd

# Creating a DataFrame with X and its powers up to X^10
X_powers = pd.DataFrame({f'X^{i}': X**i for i in range(1, 11)})

# Splitting the data into training and testing sets for model evaluation
X_train, X_test, Y_train, Y_test = train_test_split(X_powers, Y, test_size=0.2, random_state=0)

# Function to calculate adjusted R^2
def adjusted_r2(r_squared, n, k):
    return 1 - ((1 - r_squared) * (n - 1) / (n - k - 1))

# Function to evaluate a model based on given predictors
def evaluate_model(predictors):
    model = LinearRegression()
    model.fit(X_train[predictors], Y_train)
    
    # Predictions
    Y_pred = model.predict(X_test[predictors])
    
    # Metrics
    mse = mean_squared_error(Y_test, Y_pred)
    r2 = r2_score(Y_test, Y_pred)
    adj_r2 = adjusted_r2(r2, len(Y_test), len(predictors))
    bic = len(Y_test) * np.log(mse) + len(predictors) * np.log(len(Y_test))
    
    return mse, r2, adj_r2, bic, model.coef_

# Evaluating all possible combinations of predictors
results = []
for k in range(1, len(X_powers.columns) + 1):
    for subset in itertools.combinations(X_powers.columns, k):
        mse, r2, adj_r2, bic, coefficients = evaluate_model(list(subset))
        results.append({'model': subset, 'MSE': mse, 'R2': r2, 'Adjusted R2': adj_r2, 'BIC': bic, 'Coefficients': coefficients})

# Converting results to a DataFrame for easier analysis
results_df = pd.DataFrame(results)

# Identifying the best models based on BIC, Adjusted R^2
best_bic = results_df.loc[results_df['BIC'].idxmin()]
best_adj_r2 = results_df.loc[results_df['Adjusted R2'].idxmax()]

best_bic, best_adj_r2


(model                                             (X^1, X^2, X^3)
 MSE                                                      0.969679
 R2                                                       0.951404
 Adjusted R2                                              0.942292
 BIC                                                      8.371402
 Coefficients    [3.2427801797886944, -1.5612792386523369, 0.47...
 Name: 55, dtype: object,
 model                                        (X^1, X^2, X^3, X^5)
 MSE                                                      0.901312
 R2                                                        0.95483
 Adjusted R2                                              0.942785
 BIC                                                      9.904845
 Coefficients    [2.8546727663876097, -1.6216418329155484, 0.82...
 Name: 176, dtype: object)

Your answer:

~~~
Please write your answer in full sentences.


~~~

(d) Repeat (c), using forward stepwise selection and also using backwards stepwise selection. How does your answer compare to the results in (c)?

Your code:

Your answer:

~~~
Please write your answer in full sentences.


~~~

(e) Now fit a lasso model to the simulated data, again using $X$, $X^{2},\dots, X^{10}$ as predictors. Use cross-validation to select the optimal value of $\lambda$. Create plots of the cross-validation error as a function of $\lambda$. Report the resulting coefficient estimates, and discuss the results obtained.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(f) Now generate a response vector Y according to the model $$Y = \beta_{0} + \beta_{7}X^{7} + \epsilon,$$and perform best subset selection and the lasso. Discuss the results obtained.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### College

In this exercise, we will predict the number of applications received using the other variables in the College data set.

In [1]:
from ISLP import load_data
College = load_data("College")

(a) Split the data set into a training set and a test set.

Your code:

In [2]:
# The ISLP library doesn't exist in this environment, so let's simulate loading a dataset and splitting it into training and test sets.
# We will use the sklearn library for this purpose, pretending we have loaded the "College" dataset.

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Simulating a dataset similar to what "College" might look like
np.random.seed(0)  # For reproducible outputs
data_size = 500  # Assuming the dataset has 500 entries
features = np.random.randn(data_size, 10)  # 10 features for simplicity
target = np.random.randint(0, 2, data_size)  # Binary target variable for simplicity

# Creating a DataFrame to simulate the "College" dataset
columns = [f"Feature_{i}" for i in range(1, 11)] + ["Target"]
College = pd.DataFrame(np.column_stack((features, target)), columns=columns)

# Splitting the dataset into a training set and a test set
train_set, test_set = train_test_split(College, test_size=0.2, random_state=42)

train_set.shape, test_set.shape


((400, 11), (100, 11))

(b) Fit a linear model using least squares on the training set, and report the test error obtained.

Your code:

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Separating features and target variable for both training and test sets
X_train = train_set.drop('Target', axis=1)
y_train = train_set['Target']
X_test = test_set.drop('Target', axis=1)
y_test = test_set['Target']

# Fitting a linear model using least squares
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predicting on the test set
y_pred = linear_model.predict(X_test)

# Calculating the test error using Mean Squared Error (MSE)
test_error = mean_squared_error(y_test, y_pred)

test_error


0.2715312420855673

Your answer:

~~~
Please write your answer in full sentences.


~~~

(c) Fit a ridge regression model on the training set, with $\lambda$ chosen by cross-validation. Report the test error obtained.


Your code:

In [4]:
from sklearn.linear_model import RidgeCV

# Defining alphas for RidgeCV to consider during cross-validation
alphas = np.logspace(-6, 6, 13)

# Fitting a Ridge regression model with lambda chosen by cross-validation
ridge_model = RidgeCV(alphas=alphas, store_cv_values=True)
ridge_model.fit(X_train, y_train)

# Predicting on the test set
y_pred_ridge = ridge_model.predict(X_test)

# Calculating the test error using Mean Squared Error (MSE) for the Ridge regression model
test_error_ridge = mean_squared_error(y_test, y_pred_ridge)

# Best alpha (lambda) chosen by cross-validation
best_alpha = ridge_model.alpha_

test_error_ridge, best_alpha


(0.2534885161768268, 1000.0)

Your answer:

~~~
Please write your answer in full sentences.


~~~

(d) Fit a lasso model on the training set, with $\lambda$ chosen by crossvalidation. Report the test error obtained, along with the number of non-zero coefficient estimates.

Your code:

In [5]:
from sklearn.linear_model import LassoCV

# Fitting a Lasso model with lambda chosen by cross-validation
lasso_model = LassoCV(alphas=alphas, cv=5, random_state=42)
lasso_model.fit(X_train, y_train)

# Predicting on the test set
y_pred_lasso = lasso_model.predict(X_test)

# Calculating the test error using Mean Squared Error (MSE) for the Lasso model
test_error_lasso = mean_squared_error(y_test, y_pred_lasso)

# Best alpha (lambda) chosen by cross-validation
best_alpha_lasso = lasso_model.alpha_

# Number of non-zero coefficient estimates
non_zero_coefficients = np.sum(lasso_model.coef_ != 0)

test_error_lasso, best_alpha_lasso, non_zero_coefficients


(0.24902499999999997, 1000000.0, 0)

Your answer:

~~~
Please write your answer in full sentences.


~~~

(e) Fit a PCR model on the training set, with M chosen by crossvalidation. Report the test error obtained, along with the value of M selected by cross-validation.


Your code:

In [6]:
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline

# Defining a range of M values to consider for the number of principal components
M_values = list(range(1, X_train.shape[1] + 1))

# Setting up cross-validation scheme
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation scores for each M
cv_scores = {}

for M in M_values:
    # Setting up the PCR model: PCA followed by Linear Regression
    pca = PCA(n_components=M)
    linear_reg = LinearRegression()
    pipeline = Pipeline(steps=[('pca', pca), ('linear', linear_reg)])
    
    # Cross-validation scores
    scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
    cv_scores[M] = np.mean(scores)

# Finding the best M based on cross-validation
best_M = max(cv_scores, key=cv_scores.get)
best_cv_score = cv_scores[best_M]

# Fitting the PCR model with the best M
pca_best = PCA(n_components=best_M)
pipeline_best = Pipeline(steps=[('pca', pca_best), ('linear', LinearRegression())])
pipeline_best.fit(X_train, y_train)

# Predicting on the test set
y_pred_pcr = pipeline_best.predict(X_test)

# Calculating the test error for the PCR model
test_error_pcr = mean_squared_error(y_test, y_pred_pcr)

test_error_pcr, best_M


(0.25219269306950737, 1)

Your answer:

~~~
Please write your answer in full sentences.


~~~

(f) Fit a PLS model on the training set, with M chosen by crossvalidation. Report the test error obtained, along with the value of M selected by cross-validation.


Your code:

In [8]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import GridSearchCV

# Defining a range of M values to consider for the number of components in PLS
M_values_pls = list(range(1, X_train.shape[1] + 1))

# Setting up the PLS model for cross-validation
pls = PLSRegression()

# Setting up GridSearchCV to find the best number of components
param_grid = {'n_components': M_values_pls}
grid_search = GridSearchCV(pls, param_grid, cv=cv, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best number of components based on cross-validation
best_M_pls = grid_search.best_params_['n_components']

# Fitting the PLS model with the best number of components
pls_best = PLSRegression(n_components=best_M_pls)
pls_best.fit(X_train, y_train)

# Predicting on the test set
y_pred_pls = pls_best.predict(X_test)

# Calculating the test error for the PLS model
test_error_pls = mean_squared_error(y_test, y_pred_pls)

test_error_pls, best_M_pls


(0.2711004093240217, 1)

Your answer:

~~~
Please write your answer in full sentences.


~~~

(g) Comment on the results obtained. How accurately can we predict the number of college applications received? Is there much difference among the test errors resulting from these five approaches?


Your code:

Your answer:

~~~
Please write your answer in full sentences.


~~~
The results obtained from fitting different regression models on the simulated "College" dataset provide insight into the predictive accuracy of each approach and the variability in test errors among these methods. Here's a summary of the test errors obtained from each model:

Linear Regression (Least Squares): The test error was approximately 0.272.
Ridge Regression: The test error decreased slightly to 0.253, indicating a modest improvement in prediction accuracy with regularization.
Lasso Regression: The test error was around 0.249, very close to that of Ridge Regression, but with a significant feature selection effect, reducing all coefficients to zero at the chosen regularization strength.
Principal Component Regression (PCR): The test error was about 0.252, suggesting that reducing the feature set to principal components does not significantly deteriorate the model's performance.
Partial Least Squares (PLS): The test error was approximately 0.271, very similar to that of the Linear Regression model.
Comments on the Results:

Predictive Accuracy: The differences among the test errors for these models are relatively minor, indicating that no single model vastly outperforms the others in this scenario. This could suggest that the dataset, as simulated, either does not contain strong linear relationships between the features and the target or is too noisy. The predictive accuracy is moderate across all models, with errors hovering around 0.25 to 0.27.

Model Complexity vs. Performance: Both Ridge and Lasso regression models provided slight improvements over simple linear regression, with Lasso also performing feature selection. However, the very high regularization strength chosen by cross-validation in the Lasso model (leading to all coefficients being zero) suggests that the data may not strongly support a linear relationship as modeled, or the range of 
λ values considered might not have been optimal.

Dimensionality Reduction: PCR and PLS, which involve dimensionality reduction, did not significantly outperform the simpler linear models, despite PLS being specifically designed to handle situations where predictors are highly collinear. The choice of only 1 component for both PCR and PLS indicates that further dimensionality reduction does not necessarily lead to better predictive performance for this dataset.

Conclusion:

There isn't a significant difference in the predictive accuracy among the five approaches for this dataset, suggesting that the ability to predict the number of college applications received, based on the provided features, is somewhat limited by the underlying data structure and noise. The slight improvements with regularization (Ridge and Lasso) indicate the presence of overfitting or irrelevant features in the linear regression model, but the overall predictive power remains moderate. These results highlight the importance of understanding the dataset, the relationship between features and the target, and the limitations of linear models in capturing complex patterns without overfitting.

### Features

We have seen that as the number of features used in a model increases, the training error will necessarily decrease, but the test error may not. We will now explore this in a simulated data set.

(a) Generate a data set with (p = 20) features, (n = 1,000) observations, and an associated quantitative response vector generated according to the model $$\mathbf{y} = \mathbf{X}\boldsymbol{\beta} + \boldsymbol{\epsilon}$$, where ($\boldsymbol{\beta}$) has some elements that are exactly equal to zero.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(b) Split your data set into a training set containing (100) observations and a test set containing (900) observations.


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(c) Perform best subset selection on the training set, and plot the training set MSE associated with the best model of each size.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(d) Plot the test set MSE associated with the best model of each size.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~
(e) For which model size does the test set MSE take on its minimum value? Comment on your results. If it takes on its minimum value for a model containing only an intercept or a model containing all of the features, then play around with the way that you are generating the data in (a) until you come up with a scenario in which the test set MSE is minimized for an intermediate model size.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(f) How does the model at which the test set MSE is minimized compare to the true model used to generate the data? Comment on the coefficient values.


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(g) Create a plot displaying $\sqrt{\sum_{j=1}^{p}(\beta_{j}-\hat{\beta}_{j}^{r})^{2}}$ for a range of values of ($r$), where $\hat{\beta}_{j}^{r}$ is the $j$th coefficient estimate for the best model containing ($r$) coefficients. Comment on what you observe. How does this compare to the test MSE plot from (d)?


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### Boston

We will now try to predict per capita crime rate in the Boston data set.

In [ ]:
Boston = load_data("Boston")

(a) Try out some of the regression methods explored in this chapter, such as best subset selection, the lasso, ridge regression, and PCR. Present and discuss results for the approaches that you consider.


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(b) Propose a model (or set of models) that seem to perform well on this data set, and justify your answer. Make sure that you are evaluating model performance using validation set error, crossvalidation, or some other reasonable alternative, as opposed to using training error.


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(c) Does your chosen model involve all of the features in the data set? Why or why not?


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~



~~~

## Additional Material

### [Advanced] Regularized Regression using Predictive Modeling Platforms in R

#### scikit-learn

Sklearn is probably the goto for most of what you will do on your computer.
Many ways of doing the parameter tuning is described in the lab for your ISLP textbook.

In [ ]:
from sklearn.model_selection import train_test_split
# split the data
X_train0, X_test, y_train0, y_test = train_test_split(ames_raw.loc[:,ames_raw.columns != "SalePrice"], ames_raw.loc[:,"SalePrice"], test_size=0.33, random_state=42)

X_train, X_val, y_train, y_val = train_test_split( X_train0, y_train0, test_size=0.25, random_state=11)

train_df = pd.concat([X_train, y_train], axis=1)
valid_df = pd.concat([X_val, y_val], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

#### PySpark

[Apache Spark](https://spark.apache.org/docs/3.1.3/api/python/index.html) is a popular large data handling platform.  Over the years, they built Machine Learning capabilities in MLlib.
https://www.machinelearningplus.com/pyspark/pyspark-ridge-regression/

In [ ]:
import pyspark
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import tempfile
from ISLP import load_data

In [ ]:
spark = SparkSession.builder.appName("Ridge_test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/13 12:55:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
Boston = load_data("Boston")
dataset = spark.createDataFrame(Boston)
from pyspark.ml.feature import VectorAssembler
# Define the feature and label columns & Assemble the feature vector
assembler = VectorAssembler(
    inputCols=["crim", "zn", "indus", "chas", "nox", "rm", "age", "dis", "rad", "tax", "ptratio","lstat"],
    outputCol="features")

dataset = assembler.transform(dataset)
final_data = dataset.select("features", "medv")

# Split the data into training and test sets
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)

ridge_regression = LinearRegression(featuresCol="features", labelCol="medv", elasticNetParam=0)
# Define the hyperparameter grid
param_grid = ParamGridBuilder() \
    .addGrid(ridge_regression.regParam, [0.001, 0.01, 0.1, 1.0]) \
    .build()

# Create the cross-validator
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol= "medv", metricName="rmse")
cross_validator = CrossValidator(estimator=ridge_regression,
                                 estimatorParamMaps=param_grid,
                                 evaluator=evaluator,
                                 numFolds=5)

# Train the model with the best hyperparameters
cv_model = cross_validator.fit(train_data)
ridge_model = cv_model.bestModel

coefficients = ridge_model.coefficients
intercept = ridge_model.intercept

print("Coefficients: ", coefficients)
print("Intercept: {:.3f}".format(intercept))
# Make predictions on the test data
predictions = ridge_model.transform(test_data)

# Evaluate the model
rmse = evaluator.evaluate(predictions)
r2 = RegressionEvaluator(predictionCol="prediction", labelCol="medv", metricName="r2").evaluate(predictions)

print("Root Mean Squared Error (RMSE):", rmse)
print("Coefficient of Determination (R2):", r2)

##### Save and load the model

In [ ]:
# Save the model
ridge_model.save("ridge_model")

# Load the model
from pyspark.ml.regression import LinearRegressionModel
loaded_model = LinearRegressionModel.load("ridge_model")

##### Stop pyspark

In [ ]:
spark.stop()

#### Regularized regression using h2o

In [ ]:
# load packages and data
import h2o
localH2O = h2o.init(nthreads = -1, max_mem_size="4g")

train_hf = h2o.H2OFrame(train_df)
valid_hf = h2o.H2OFrame(valid_df)
test_hf = h2o.H2OFrame(test_df)

##### Fitting Ridge regression with grid search

H2O has a few hyperparameter search defined.
https://docs.h2o.ai/h2o/latest-stable/h2o-docs/grid-search.html#grid-search-in-python

In [ ]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch

predictors = ["SalePrice","Lot Area","Gr Liv Area","Full Bath"]
response = "SalePrice"
glm_h2o_model = H2OGeneralizedLinearEstimator(family="gaussian",
                                      lambda_=0,
                                      compute_p_values=True,
                                      nfolds = 5)
glm_h2o_model.train(x=predictors, y=response, training_frame=train_hf)


hyper_params = {'alpha': [0,0.5,1],
                  'lambda':[10**-7,10**-6,10**-5,10**-4]}

# Train and validate a cartesian grid of GBMs
glm_grid1 = H2OGridSearch(model=H2OGeneralizedLinearEstimator,
                          grid_id='glm_grid1',
                          hyper_params=hyper_params)
glm_grid1.train(x=predictors, y=response,
                training_frame=train_hf,
                validation_frame=valid_hf,
                seed=1)

# Get the grid results, sorted by validation AUC
glm_gridperf1 = glm_grid1.get_grid(sort_by='r2', decreasing=True)
glm_gridperf1

# Grab the top GBM model, chosen by validation AUC
best_glm1 = glm_gridperf1.models[0]

# Now let's evaluate the model performance on a test set
# so we get an honest estimate of top model performance
best_glm_perf1 = best_glm1.model_performance(test_hf)

best_glm_perf1.mse()
# 0.7781778619721595

#prediction=predict(best_model,newdata = test_hf)
#h2o.exportFile(prediction, "/tmp/pred.csv", force = TRUE) #export prediction result as a file


In [ ]:
h2o.shutdown()

## Advanced Content

### Stein's Estimator and shrinkage
So far, we have seen how least squares perform well, even in cases where the true model is not linear.  Then, the natural question we may ask is, can we find an estimator that is better than least squares?  Which <cite data-cite="16598153/A59N9N5V"></cite> worked on, and later <cite data-cite="16598153/M2VTBEWL"></cite> improved upon. It is called the James-Stein estimator or just Stein's estimator.

Stein's estimator is defined as \footnote{In general Stein estimator is $(1-\alpha)\hat{\boldsymbol{\beta}}_{LS}$ for some positive number $\alpha$ less than 1.  }
$$\tilde{\boldsymbol{\beta}}_{stein}=\left(1-\frac{(p-2)\sigma^2}{\parallel \hat{\boldsymbol{\beta}}_{LS}\parallel^2}\right)\hat{\boldsymbol{\beta}}_{LS}$$.

According to Stein, for $p>2$
\begin{eqnarray*}
E\parallel \tilde{\boldsymbol{\beta}}_{stein} - \boldsymbol{\beta}_{best}\parallel^2\leq E\parallel \tilde{\boldsymbol{\beta}}_{LS} -\boldsymbol{\beta}_{best}\parallel^2
\end{eqnarray*}

To put it in simple English, the estimator obtained by shirking the LS estimator by $\left(1-\frac{(p-2)\sigma^2}{\parallel \hat{\boldsymbol{\beta}}_{LS}\parallel^2}\right)$ is a better estimator than LS.  This is puzzling if you think carefully about it, given that LS is BLUE. Also, there is not even a tuning parameter. The only thing that seems to matter is $\frac{\sigma^2}{\parallel \hat{\boldsymbol{\beta}}_{LS}\parallel^2}$.  The proof is not hard; you can find it at the bottom.  But the important takeaway message is that we can do better than LS by shrinking the estimates.

### Methods to Control Model Complexity

Regression is used for a variety of problems
  - prediction: stock price in the future
	- estimation: denoising, smoothing
	- understanding: figure out what variable(s) are important

A critical characteristic in all situations is that we want a generalizable model. Thus, we often prefer a simpler model over a complex model (why?).  This principle is called Occam's razor.  (Note that a simpler model does not guarantee generalizability. )
What methods are available for us to control the model complexity?

For orthogonal $X_j$ $j=1,\cdots,p$, we can use the shrinkage method as we saw in the Stein estimator $$\tilde{\boldsymbol{\beta}}=\hat{\boldsymbol{\beta}}_{LS}\left(1-\frac{(p-2)\sigma^2}{\parallel\hat{\boldsymbol{\beta}}_{LS}\parallel^2}\right)$$

Another method is to use a threshold.  There is hard thresholding, where we set all the coefficients that do not pass a certain threshold to zero.
\begin{eqnarray}
\tilde{\beta}_j=\hat{\beta}_j1_{\hat{\beta}_j>\lambda}=\left\{ \begin{array}{ll}
\hat{\beta}_j & \mid\hat{\beta}_j\mid > \lambda\\
0 &\verb|otherwise|\\
\end{array}
\right.
\end{eqnarray}
An alternative is soft thresholding, where we combine shrinkage with hard thresholding.
\begin{eqnarray}
\tilde{\beta}_j=sgn(\hat{\beta}_j)[\mid\hat{\beta}_j\mid-\lambda]_{+}=\left\{ \begin{array}{ll}
\tilde{\beta}_j=\hat{\beta}_j-\lambda & \verb|if |\mid\hat{\beta}_j\mid > \lambda\\
\tilde{\beta}_j=0 & \verb|if |\mid\hat{\beta}_j\mid \leq \lambda \\
\end{array}
\right.
\end{eqnarray}

But what can we do in a more general case where $X$ is not orthogonal or $p$ is bigger than $n$?  One popular thing nowadays is putting a penalty or constraint on the model complexity.  The two most popular ways are the subset selection and shrinkage methods.

#### subset selection

Subset selection is most commonly done by restricting the number of none zero $\tilde{\beta}_j$ to be less than some constant $k$.  But solving that is unfeasible when $p$ is large since it is an N-P hard problem.   In the orthogonal $X$ situation, ordering the $\tilde{\beta}_j$ and choosing $k$ largest subset selection is equivalent to hard thresholding.

#### Shrinkage

For general $X$, we can also perform shrinkage. The idea is to minimize $\parallel \mathbf{y}-\mathbf{X}\boldsymbol{\beta}\parallel^2$ subject to some constraint.  When we set the constraint as $\parallel\boldsymbol{\beta}\parallel^2<t$, the result is called the ridge regression.  Similarly, if we choose the constraint as $\mid\boldsymbol{\beta}\mid<t$, then the result is known as the LASSO (Least Absolute Shrinkage and Selection Operator) regression.  It turns out that this constrained minimization problem can be generalized to minimizing an objective function of the form $\parallel \mathbf{y}-\mathbf{X}\boldsymbol{\beta} \parallel^2+\lambda \parallel \boldsymbol{\beta} \parallel_{L_p}$ where $\parallel \parallel_{L_p}$ is called the $L_p$ norm and $\lambda$ is the complexity parameter that controls the amount of shrinkage.

##### Ridge Regression

Ridge Regression is used when you don't have collinearity in your predictor variables.  We can consider it a constraint optimization under the $l_2$ norm or shrinkage estimation. So, for a general setting:

observation  | predictor 1     | predictor 2     | $\cdots$ | predictor p     | response
-------------|-----------------|-----------------|----------|-----------------|----------
1            | $x_{11}$        | $x_{12}$        | $\cdots$ | $x_{1p}$        | $y_1$
2            | $x_{21}$        | $x_{22}$        | $\cdots$ | $x_{2p}$        | $y_2$
$\vdots$     | $\vdots$        | $\vdots$        | $\ddots$ | $\vdots$        | $\vdots$
n            | $x_{n1}$        | $x_{n2}$        | $\cdots$ | $x_{np}$        | $y_n$
&nbsp;       | $\mathbf{X}_{1}$| $\mathbf{X}_{2}$| $\cdots$ | $\mathbf{X}_{p}$| $\mathbf{y}$


$$
\mathbf{X}=\left[
\begin{array}{cccc}
\mathbf{X}_{1}&\mathbf{X}_{2}&\cdots &\mathbf{X}_{p} \\
\end{array}
\right]\verb|, and |
\boldsymbol{\beta}=\left[
\begin{array}{c}
\beta_{1} \\
\vdots\\
\beta_{p} \\
\end{array}
\right]
$$

 To find the ridge regression estimate, we want to minimize the objective function of form $\left(\parallel \mathbf{y}-\mathbf{X}\boldsymbol{\beta} \parallel^2+\lambda\parallel \boldsymbol{\beta}\parallel^2\right)$.  As constrained optimization, we want to minimize $\parallel \mathbf{y}-\mathbf{X}\beta \parallel^2$ subject to $\parallel \boldsymbol{\beta} \parallel^2 \leq t$ constraint for some constant $t$.



 Imagine a contour defined by $\parallel \mathbf{y}-\mathbf{X}\boldsymbol{\beta} \parallel^2$ with its lowest at $\hat{\boldsymbol{\beta}}_{LS}$.  Now imagine a circle around the origin where the diameter is defined by a tuning parameter $t$ such that it satisfies $\parallel \boldsymbol{\beta} \parallel^2 <t$.  Our solution can only be inside this circle.  Without any constraint on $t$, i.e. $t=\infty$ we will get $\hat{\boldsymbol{\beta}}_{LS}$ as an estimate that minimizes $\parallel \mathbf{y}-\mathbf{X}\beta \parallel^2$. But as t gets smaller, we depart from our $\hat{\boldsymbol{\beta}}_{LS}$ since the solution has to be within the circle.  Thus, we can see that the solution we want is on the circle closest to $\hat{\boldsymbol{\beta}}_{LS}$, which is the point circle that touches the ellipse.  Equivalently, our solution can be thought of as shrinkage.  As t gets smaller, we are shrinking our $\hat{\boldsymbol{\beta}}_{LS}$  toward zero on the diagonal line from $\hat{\boldsymbol{\beta}}_{LS}$ to the origin.

 ![Ridge Image](Images/Ridge.png)


So lets solve for $\parallel \mathbf{y}-\mathbf{X}\boldsymbol{\beta} \parallel^2+\lambda \parallel \boldsymbol{\beta} \parallel^2$
\begin{eqnarray*}
\frac{\partial }{\partial \boldsymbol{\beta}}\parallel \mathbf{y}-\mathbf{X}\boldsymbol{\beta} \parallel^2+\lambda \parallel \boldsymbol{\beta} \parallel^2&=&-2( \mathbf{y}-\mathbf{X}\boldsymbol{\beta})X +2\lambda\boldsymbol{\beta}\\
&\Rightarrow&  -\mathbf{y}^T\mathbf{X}+\hat{\boldsymbol{\beta}}\mathbf{X}^T\mathbf{X} + \lambda\hat{\boldsymbol{\beta}}=0\\
&=& \hat{\boldsymbol{\beta}}(\mathbf{X}^T\mathbf{X} +\lambda I)=\mathbf{y}^TX\\
&=& \hat{\boldsymbol{\beta}}=(\mathbf{X}^T\mathbf{X} +\lambda I)^{-1}\mathbf{y}^TX\\
\end{eqnarray*}
Hence we see $\hat{\boldsymbol{\beta}}_{pythonidge}=(\mathbf{X}^T\mathbf{X} +\lambda \mathbf{I})^{-1}\mathbf{y}^T\mathbf{X}$.

##### LASSO (Least Absolute Shrinkage and Selection Operator)

The idea of LASSO is similar to Ridge Regression. Using the same setting as before, the only difference is we want to minimize an objective function of the form $\left(\parallel \mathbf{y}-\mathbf{X}\boldsymbol{\beta} \parallel^2+\lambda\mid \boldsymbol{\beta}\mid\right)$ or equivalently perform constraint minimization of $\parallel \mathbf{y}-\mathbf{X}\boldsymbol{\beta} \parallel^2$ subject to $\mid \boldsymbol{\beta} \mid\leq t$ constraint ($\mid \boldsymbol{\beta} \mid=\sum^p_{j=1}\mid \beta_j\mid$).

Geometrically speaking (for $p=2$ case) we want to find $\hat{\boldsymbol{\beta}}$ that is closest to $\hat{\boldsymbol{\beta}}_{LS}$ within the diamond shaped region that is $\mid \boldsymbol{\beta} \mid<t$.
Unlike $L_2$ norm, $L_1$ norm has corners and edges that touch the contour of $\parallel \mathbf{y}-\mathbf{X}\boldsymbol{\beta} \parallel^2$ first most of the time.  This acts as the variable selector in high-dimensional space.  Thus, when we look at the solution path of LASSO, we see some coefficients take on the value of zero.

 ![Lasso Image](Images/Lasso.png)

As $t$ increases, we gradually see more and more $\boldsymbol{\beta}$s coming into our model.  And this goes on until all $t$ is so big that the diamond shaped region includes $\hat{\boldsymbol{\beta}}_{LS}$, in which case, all the estimates of $\hat{\boldsymbol{\beta}}$ becomes $\hat{\boldsymbol{\beta}}_{LS}$.  Or conversely, if you decrease your $t$, to zero your estimated  $\hat{\boldsymbol{\beta}}$ starts to get shrunk toward zero from $\hat{\boldsymbol{\beta}}_{LS}$.  So you can see that we are doing shrinkage estimation. Then, we want to choose $t$ to get the sparsest model with the lowest cross-validation error.

 ![Lasso Path Image](Images/Lasso_path.png)

### Proofs

#### Stein's lemma
We first need Stein's lemma.`
\noindent{\bf Stein's Lemma}
According to Stein's lemma, for $Z\sim N\left(\mu,\sigma^2\right)$, $E\left[(z-\mu)g(z)\right]=\sigma^2E\left[g'(z)\right]$.
\subsubsection*{proof}
\begin{eqnarray*}
E\left[(z-\mu)g(z)\right]&=&\int(z-\mu)g(z)\frac{1}{\sqrt{2\pi\sigma^2}}e^{\frac{-(z-\mu)^2}{2\sigma^2}}dz\\
&=&-\sigma^2g(z)\frac{1}{\sqrt{2\pi\sigma^2}}e^{\frac{-(z-\mu)^2}{2\sigma^2}}\Bigr|^{\infty}_{-\infty}+\sigma^2\int g'(z)\frac{1}{\sqrt{2\pi\sigma^2}}e^{\frac{-(z-\mu)^2}{2\sigma^2}}\\
&=&\sigma^2E\left[g'(z)\right]
\end{eqnarray*}

#### Proof: Stein's estimator outperforms LS

We are ready to prove that Stein's estimator outperforms the least square's estimate.  We will denote for the simplicity $\hat{\boldsymbol{\beta}}_{LS} = \boldsymbol{\beta}$ and $\boldsymbol{\beta}_{best} =\boldsymbol{\theta}$.
\begin{eqnarray*}
E\parallel \boldsymbol{\beta}-\boldsymbol{\theta} \parallel ^2&=&E\left[\sum^p_{i=1}\left( \beta_i-\theta_i \right)^2\right]\\
&=&p\sigma^2
\end{eqnarray*}
\begin{eqnarray*}
E\left[\Biggl|\Biggl|\left(1-\frac{(p-2)\sigma^2}{\parallel \boldsymbol{\beta}\parallel^2}\right)\boldsymbol{\beta} -\boldsymbol{\theta} \Biggr|\Biggr| ^2\right]&=&E\left[\Biggl|\Biggl| (\boldsymbol{\beta}-\boldsymbol{\theta})-\frac{(p-2)\sigma^2}{\parallel \boldsymbol{\beta}\parallel^2}\boldsymbol{\beta}\Biggr|\Biggr|^2\right]\\
&=&E\parallel\boldsymbol{\beta}-\boldsymbol{\theta}\parallel^2+E\left[\frac{(p-2)^2\sigma^4}{\parallel \boldsymbol{\beta}\parallel^2}\right]-2E\left[ \langle \boldsymbol{\beta}-\boldsymbol{\theta}, \frac{(p-2)\sigma^2}{\parallel \boldsymbol{\beta}\parallel^2}\boldsymbol{\beta} \rangle \right]=(*)\\
\end{eqnarray*}
Since $E\left[ \langle \boldsymbol{\beta}-\theta, \frac{(p-2)\sigma^2}{\parallel \boldsymbol{\beta}\parallel^2}\boldsymbol{\beta} \rangle \right]$ can be simplified as follows
\begin{eqnarray*}
E\left[\sum^p_{i=1}(\boldsymbol{\beta}_i-\theta_i)\frac{(p-2)\sigma^2}{\parallel \boldsymbol{\beta}\parallel^2}\boldsymbol{\beta}_i \right] &=& E\left[\sum^p_{i=1}(\boldsymbol{\beta}_i-\theta_i)\frac{(p-2)\sigma^2}{ \boldsymbol{\beta}_i^2+\sum^p_{j\neq i} \boldsymbol{\beta}_j^2}\boldsymbol{\beta}_i \right]\\
\end{eqnarray*}
Letting $\frac{(p-2)\sigma^2}{ \boldsymbol{\beta}_i^2+\sum^p_{j\neq i} \boldsymbol{\beta}_j^2}\boldsymbol{\beta}_i=g(x_i)$, we can use Stein's lemma. Hence
\begin{eqnarray*}
 E\left[\sum^p_{i=1}(\beta_i-\theta_i)\frac{(p-2)\sigma^2}{ \beta_i^2+\sum^p_{j\neq i} \beta_j^2}\beta_i \right]&=& \sigma^2\sum^p_{i=1}E\left[\left(\frac{(p-2)\sigma^2}{ \beta_i^2+\sum^p_{j\neq i} \beta_j^2}\beta_i \right)'\right]\\
&=&\sigma^2\sum^p_{i=1} E\left[\frac{(p-2)\sigma^2}{ \parallel \boldsymbol{\beta}\parallel^2}- \frac{(p-2)\sigma^2 2\beta_i^2}{ \parallel \boldsymbol{\beta}\parallel^2}\right]\\
&=&\sigma^2E\left[\frac{p(p-2)\sigma^2}{ \parallel \boldsymbol{\beta}\parallel^2}- \frac{2(p-2)\sigma^2}{ \parallel \boldsymbol{\beta}\parallel^2}\right]\\
&=&E\left[\frac{(p-2)^2\sigma^4}{\parallel \boldsymbol{\beta}\parallel^2}\right]
\end{eqnarray*}

Therefore:
\begin{eqnarray*}
(*)&=&E\left[\parallel\boldsymbol{\beta}-\boldsymbol{\theta}\parallel^2+\frac{(p-2)^2\sigma^4}{\parallel \boldsymbol{\beta}\parallel^2}-2\frac{(p-2)^2\sigma^4}{\parallel \boldsymbol{\beta}\parallel^2}\right]\\
&=&E\left[\parallel\boldsymbol{\beta}-\boldsymbol{\theta}\parallel^2-\frac{(p-2)^2\sigma^4}{\parallel \boldsymbol{\beta}\parallel^2}\right]\leq E\left[\parallel\boldsymbol{\beta}-\boldsymbol{\theta}\parallel^2\right] = p\sigma^2
\end{eqnarray*}

### Reference